In [13]:
import vtk
import numpy as np

mbf = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/MBFIschemicRegion_cutoff90.0.vtu"

def ReadVTU(file):
    reader = vtk.vtkXMLUnstructuredGridReader()
    reader.SetFileName(file)
    reader.Update()
    return reader.GetOutput()

MBF = ReadVTU(mbf)

points = MBF.GetPoints()
point_coords = np.array([points.GetPoint(i) for i in range(points.GetNumberOfPoints())])
#print(point_coords)



In [14]:
mean_center = np.mean(point_coords, axis=0)
cov_matrix = np.cov(point_coords.T)
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

sorted_indices = np.argsort(-eigenvalues)
long_axis = eigenvectors[:, sorted_indices[0]]
short_axis1 = eigenvectors[:, sorted_indices[1]]
short_axis2 = eigenvectors[:, sorted_indices[2]]

min_bounds = np.min(point_coords, axis=0)
max_bounds = np.max(point_coords, axis=0)
size_x = (max_bounds[0] - min_bounds[0])
size_y = (max_bounds[1] - min_bounds[1])
size_z = (max_bounds[2] - min_bounds[2])

# Function to create a plane from two spanning vectors
def create_plane(center, vec1, vec2, size1, size2):
    """Creates a VTK plane polydata given a center, two basis vectors, and sizes."""
    plane_points = vtk.vtkPoints()
    plane_cells = vtk.vtkCellArray()

    # Compute corners of the plane
    v1_scaled = vec1 * size1 / 2
    v2_scaled = vec2 * size2 / 2

    corners = [
        center - v1_scaled - v2_scaled,
        center + v1_scaled - v2_scaled,
        center + v1_scaled + v2_scaled,
        center - v1_scaled + v2_scaled,
    ]

    for i, corner in enumerate(corners):
        plane_points.InsertNextPoint(corner)

    # Create a quad (polygon)
    quad = vtk.vtkPolygon()
    quad.GetPointIds().SetNumberOfIds(4)
    for i in range(4):
        quad.GetPointIds().SetId(i, i)

    plane_cells.InsertNextCell(quad)

    # Create polydata
    plane_polydata = vtk.vtkPolyData()
    plane_polydata.SetPoints(plane_points)
    plane_polydata.SetPolys(plane_cells)

    return plane_polydata

# Create short-axis and long-axis planes
short_axis_plane = create_plane(mean_center, short_axis1, short_axis2, size_x, size_y)
long_axis_plane = create_plane(mean_center, long_axis, short_axis1, size_x, size_z)



#plane1 = "/Users/ana/Documents/AnahitaSeresti/pshortaxis.vtp"
#plane2 = "/Users/ana/Documents/AnahitaSeresti/plongaxis.vtp"

def WriteVTPFile(FileName,Data):
    writer=vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(FileName)
    writer.SetInputData(Data)
    writer.Update()

#WriteVTPFile(plane1, short_axis_plane)
#WriteVTPFile(plane2, long_axis_plane)




In [15]:
def ReadVTP(FileName):
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(FileName)
    reader.Update()
    return reader.GetOutput()

InputFileName = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/RayBasedIschemicProfile_2"

def WriteVTUFile(FileName, Data):
    writer = vtk.vtkXMLUnstructuredGridWriter()
    writer.SetFileName(FileName)
    writer.SetInputData(Data)
    writer.Update()

import glob
SliceFileName = glob.glob(f"{InputFileName}/Slice*")
SliceFileName = sorted(SliceFileName)

Slice = []
for file in SliceFileName:
    slice_ = ReadVTP(file)
    Slice.append(slice_)


In [16]:
AnnotationPoints = []

def GetCentroid(Surface):
	Centroid=vtk.vtkCenterOfMass()
	Centroid.SetInputData(Surface)
	Centroid.SetUseScalarsAsWeights(False)
	Centroid.Update()
	return Centroid.GetCenter()



for slice in Slice:
    AnnotationPoints.append(GetCentroid(slice))

res = 5
def Line(point1, point2, res):
    line = vtk.vtkLineSource()
    line.SetPoint1(point1)
    line.SetPoint2(point2)
    line.SetResolution(res)
    line.Update()
    return line.GetOutput()



def BoldLine(centerline):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(centerline)
    tube_filter.SetRadius(1)  # Adjust this value to change thickness
    tube_filter.SetNumberOfSides(50)  # Higher = smoother tube
    tube_filter.CappingOn()  # Close tube ends
    tube_filter.Update()
    return tube_filter.GetOutput()

CenterLine = vtk.vtkAppendPolyData()
tangent = []
print(len(AnnotationPoints))
for i in range(len(AnnotationPoints)-1):
    print(i)
    CL_ = Line(AnnotationPoints[i], AnnotationPoints[i+1], res)
    #print(CL_.GetNumberOfPoints())
    CenterLine.AddInputData(CL_)
    tg_ = np.array([(AnnotationPoints[i+1][0] - AnnotationPoints[i][0]), (AnnotationPoints[i+1][1] - AnnotationPoints[i][1]), (AnnotationPoints[i+1][2] - AnnotationPoints[i][2])])
    #print(tg_)
    for _ in range(res+2):
        tangent.append(tg_)

print(len(tangent))


CenterLine.Update()



line_path = f"{InputFileName}/CenterLine.vtp"
WriteVTPFile(line_path, BoldLine(CenterLine.GetOutput()))


2
0
7


In [19]:
InputFolder = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/RayBasedIschemicProfile_2"

point1 = np.array([mean_center[0] + short_axis1[0]*40, mean_center[1] + short_axis1[1]*40, mean_center[2] + short_axis1[2]*40])
point2 = np.array([mean_center[0] - short_axis1[0]*30, mean_center[1] - short_axis1[1]*30, mean_center[2] - short_axis1[2]*30])

ShortAxis1 = Line(point1, point2, 15)
shortaxis1_path = InputFolder + "/shortaxis1.vtp"
WriteVTPFile(shortaxis1_path, ShortAxis1)

point1 = np.array([mean_center[0] + short_axis2[0]*35, mean_center[1] + short_axis2[1]*35, mean_center[2] + short_axis2[2]*35])
point2 = np.array([mean_center[0] - short_axis2[0]*35, mean_center[1] - short_axis2[1]*35, mean_center[2] - short_axis2[2]*35])

ShortAxis2 = Line(point1, point2, 15)
shortaxis2_path = InputFolder + "/shortaxis2.vtp"
WriteVTPFile(shortaxis2_path, ShortAxis2)


In [ ]:
def CutPlane(Volume,Origin,Norm):
    plane=vtk.vtkPlane()
    plane.SetOrigin(Origin)
    plane.SetNormal(Norm)
    Slice=vtk.vtkCutter()
    Slice.GenerateTrianglesOff()
    Slice.SetCutFunction(plane)
    Slice.SetInputData(Volume)
    Slice.Update()
    return Slice.GetOutput()


points = CenterLine.GetOutput().GetPoints()
print(points.GetNumberOfPoints())
center_points = np.array([points.GetPoint(i) for i in range(points.GetNumberOfPoints())])

Myo_slice = CutPlane(MBF, center_points[3], tangent[0])
slice_path = f"{InputFileName}/ClippedSlice.vtp"
WriteVTPFile(slice_path, Myo_slice)

print(len(center_points))
print(len(tangent))

for i in range(points.GetNumberOfPoints()):
    Myo_slice = CutPlane(MBF, center_points[i], tangent[i])
    slice_path = f"{InputFileName}/ClippedSlice_{i}.vtp"
    WriteVTPFile(slice_path, Myo_slice)

In [ ]:
normal = tangent[0]
ray_ = np.array([-normal[2], 0, normal[0]])

print(np.dot(normal, ray_))
ray_ /= np.linalg.norm(ray_)
origin = center_points[0]
point2 = np.array([origin[0]+ 25*ray_[0], origin[1] + 25*ray_[1], origin[2] + 25*ray_[2]])
ray = Line(origin, point2, res)

#line_path = f"{InputFileName}/ray_.vtp"
#WriteVTPFile(line_path, BoldLine(ray))

def rotate_vector(ray, normal, angle):
    angle = np.radians(angle)
    normal /= np.linalg.norm(normal)
    ray_rot = (ray*np.cos(angle) + np.cross(normal, ray) * np.sin(angle) + normal * np.dot(normal, ray) * (1 - np.cos(angle)))
    return ray_rot

angles = np.linspace(0, 360, 60, endpoint= False)
for i in range(60):
    ray_new = rotate_vector(ray_, normal, angles[i])
    point2 = np.array([origin[0]+ 50*ray_new[0], origin[1] + 50*ray_new[1], origin[2] + 50*ray_new[2]])
    ray = Line(origin, point2, res)
    line_path = f"{InputFileName}/ray_{i}.vtp"
    WriteVTPFile(line_path, BoldLine(ray))


In [ ]:
def ProbeMyocardium(InputData, SourceData):
    probe = vtk.vtkProbeFilter()
    probe.AddInputData(InputData)
    probe.SetSourceData(SourceData)
    probe.Update()
    return probe.GetOutput()

n_ray = 50
angles = np.linspace(0, 360, n_ray, endpoint= False)
for k in range(len(center_points)):
    origin = center_points[k]
    normal = tangent[k]
    all_ray = vtk.vtkAppendPolyData()
    for i in range(n_ray):
        ray_new = rotate_vector(ray_, normal, angles[i])
        point2 = np.array([origin[0]+ 50*ray_new[0], origin[1] + 50*ray_new[1], origin[2] + 50*ray_new[2]])
        ray = Line(origin, point2, res)
        ray_projected = ProbeMyocardium(ray, MBF)
        all_ray.AddInputData(BoldLine(ray_projected))
    all_ray.Update()
    line_path = f"{InputFileName}/ray_projected{k}.vtp"
    WriteVTPFile(line_path, all_ray.GetOutput())

In [ ]:
point1 = np.array([mean_center[0] + long_axis[0]*50, mean_center[1] + long_axis[1]*50, mean_center[2] + long_axis[2]*50])
point2 = np.array([mean_center[0] - long_axis[0]*50, mean_center[1] - long_axis[1]*50, mean_center[2] - long_axis[2]*50])

CL_str = Line(point1, point2, 15)
OutputFolder = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/RayBasedIschemicProfile_PCA"

WriteVTPFile(f"{OutputFolder}/CL_PCA.vtp", BoldLine(CL_str))

points = CL_str.GetPoints()
#print(points.GetNumberOfPoints())
center_points = np.array([points.GetPoint(i) for i in range(points.GetNumberOfPoints())])


for i in range(points.GetNumberOfPoints()):
    Myo_slice = CutPlane(MBF, center_points[i], long_axis)
    slice_path = f"{OutputFolder}/ClippedSlice_{i}.vtp"
    WriteVTPFile(slice_path, Myo_slice)

ray_ = np.array([-long_axis[2], 0, long_axis[0]])
n_ray = 50
angles = np.linspace(0, 360, n_ray, endpoint= False)
for k in range(len(center_points)):
    origin = center_points[k]
    normal = tangent[k]
    all_ray = vtk.vtkAppendPolyData()
    for i in range(n_ray):
        ray_new = rotate_vector(ray_, long_axis, angles[i])
        point2 = np.array([origin[0]+ 100*ray_new[0], origin[1] + 100*ray_new[1], origin[2] + 100*ray_new[2]])
        ray = Line(origin, point2, res)
        ray_projected = ProbeMyocardium(ray, MBF)
        all_ray.AddInputData(BoldLine(ray_projected))
    all_ray.Update()
    line_path = f"{OutputFolder}/ray_projected{k}.vtp"
    WriteVTPFile(line_path, all_ray.GetOutput())

In [ ]:
mbf_lower = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/RayBasedIschemicProfile_SplitMyocardium/MBF_Lower.vtu"
mbf_upper = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/RayBasedIschemicProfile_SplitMyocardium/MBF_Upper.vtu"

MBF_Lower = ReadVTU(mbf_lower)
MBF_Upper = ReadVTU(mbf_upper)

def ExtractSurface(volume):
	#Get the outer surface of the volume
	surface=vtk.vtkDataSetSurfaceFilter()
	surface.SetInputData(volume)
	surface.Update()
	return surface.GetOutput()

MBF_Lower_Surface = ExtractSurface(MBF_Lower)
MBF_Upper_Surface = ExtractSurface(MBF_Upper)


def PCA_Analysis(Volume):
    points = Volume.GetPoints()
    point_coords = np.array([points.GetPoint(i) for i in range(points.GetNumberOfPoints())])

    mean_center = np.mean(point_coords, axis=0)
    cov_matrix = np.cov(point_coords.T)
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

    sorted_indices = np.argsort(-eigenvalues)
    long_axis = eigenvectors[:, sorted_indices[0]]
    short_axis1 = eigenvectors[:, sorted_indices[1]]
    short_axis2 = eigenvectors[:, sorted_indices[2]]

    return mean_center, long_axis, short_axis1, short_axis2

center_lower, _, _, short_axis_lower = PCA_Analysis(MBF_Lower)
center_upper, _, _, short_axis_upper = PCA_Analysis(MBF_Upper)

length = 35

point1 = np.array([center_lower[0] + short_axis_lower[0]*length, center_lower[1] + short_axis_lower[1]*length, center_lower[2] + short_axis_lower[2]*length])
point2 = np.array([center_lower[0] - short_axis_lower[0]*length, center_lower[1] - short_axis_lower[1]*length, center_lower[2] - short_axis_lower[2]*length])

res = 50
CL_str_lower = Line(point1, point2, res)
OutputFolder = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/RayBasedIschemicProfile_SplitMyocardium"

point1 = np.array([center_upper[0] + short_axis_upper[0]*length, center_upper[1] + short_axis_upper[1]*length, center_upper[2] + short_axis_upper[2]*length])
point2 = np.array([center_upper[0] - short_axis_upper[0]*length, center_upper[1] - short_axis_upper[1]*length, center_upper[2] - short_axis_upper[2]*length])

CL_str_upper = Line(point1, point2, res)

WriteVTPFile(f"{OutputFolder}/CL_PCA_Lower.vtp", BoldLine(CL_str_lower))
WriteVTPFile(f"{OutputFolder}/CL_PCA_Upper.vtp", BoldLine(CL_str_upper))


points_lower = CL_str_lower.GetPoints()
center_points_lower = np.array([points_lower.GetPoint(i) for i in range(points_lower.GetNumberOfPoints())])
for i in range(points_lower.GetNumberOfPoints()):
    Myo_slice = CutPlane(MBF_Lower, center_points_lower[i], short_axis_lower)
    #slice_path = f"{OutputFolder}/ClippedSlice_Lower_{i}.vtp"
    #WriteVTPFile(slice_path, Myo_slice)

points_upper = CL_str_upper.GetPoints()
center_points_upper = np.array([points_upper.GetPoint(i) for i in range(points_upper.GetNumberOfPoints())])
for i in range(points_upper.GetNumberOfPoints()):
    Myo_slice = CutPlane(MBF_Upper, center_points_upper[i], short_axis_upper)
    #slice_path = f"{OutputFolder}/ClippedSlice_Upper_{i}.vtp"
    #WriteVTPFile(slice_path, Myo_slice)

def Cylinder(MyocardiumCenterLine):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(MyocardiumCenterLine)
    tube_filter.SetRadius(35)  # Adjust this value to change thickness
    tube_filter.SetNumberOfSides(1800)  # Higher = smoother tube
    tube_filter.CappingOff()  # Close tube ends
    tube_filter.Update()
    return tube_filter.GetOutput()

cylinder_lower = f"{OutputFolder}/CylinderLower.vtp"
Cylinder_Lower = Cylinder(CL_str_lower)


cylinder_upper = f"{OutputFolder}/CylinderUpper.vtp"
Cylinder_Upper = Cylinder(CL_str_upper)


from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk

ray_lower_ = np.array([-short_axis_lower[2], 0, short_axis_lower[0]])
n_ray = 1800
res = 50
angles = np.linspace(0, 360, n_ray, endpoint= False)
Lower_Rays = vtk.vtkAppendPolyData()
for k in range(len(center_points_lower)):
    origin = center_points_lower[k]
    all_ray = vtk.vtkAppendPolyData()
    for i in range(n_ray):
        ray_new = rotate_vector(ray_lower_, short_axis_lower, angles[i])
        point2 = np.array([origin[0]+ 100*ray_new[0], origin[1] + 100*ray_new[1], origin[2] + 100*ray_new[2]])
        ray = Line(origin, point2, res)
        ray_projected = ProbeMyocardium(ray, MBF_Lower)
        ischemic_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("IschemicTerritory"))
        is_ischemic = np.where(ischemic_profile > 0.5)[0]
        if is_ischemic.size > 0:
            ischemic_profile = numpy_to_vtk(np.array([1 for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            ischemic_profile = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))
        ischemic_profile.SetName("IschemicProfile")
        ray_projected.GetPointData().AddArray(ischemic_profile)

        territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
        territory = territory_profile[territory_profile > 0]
        if territory.size > 0:
            territory_tag = np.bincount(territory).argmax()
            territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
        territory_profile.SetName("TerritoryProfile")
        ray_projected.GetPointData().AddArray(territory_profile)
        
        all_ray.AddInputData(BoldLine(ray_projected))
    all_ray.Update()
    Lower_Rays.AddInputData(all_ray.GetOutput())
    
    #line_path = f"{OutputFolder}/ray_projected_lower{k}.vtp"
    #WriteVTPFile(line_path, all_ray.GetOutput())
Lower_Rays.Update()
#line_path = f"{OutputFolder}/ray_lower.vtp"
#WriteVTUFile(line_path, Lower_Rays.GetOutput())
MBF_Lower_Ischemic_Profile = ProbeMyocardium(MBF_Lower_Surface, Lower_Rays.GetOutput()) 



In [ ]:
ray_upper_ = np.array([-short_axis_upper[2], 0, short_axis_upper[0]])
angles = np.linspace(0, 360, n_ray, endpoint= False)
Upper_Rays = vtk.vtkAppendPolyData()
for k in range(len(center_points_upper)):
    origin = center_points_upper[k]
    all_ray = vtk.vtkAppendPolyData()
    for i in range(n_ray):
        ray_new = rotate_vector(ray_upper_, short_axis_upper, angles[i])
        point2 = np.array([origin[0]+ 100*ray_new[0], origin[1] + 100*ray_new[1], origin[2] + 100*ray_new[2]])
        ray = Line(origin, point2, res)
        ray_projected = ProbeMyocardium(ray, MBF_Upper)
        ischemic_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("IschemicTerritory"))
        is_ischemic = np.where(ischemic_profile > 0.5)[0]
        if is_ischemic.size > 0:
            ischemic_profile = numpy_to_vtk(np.array([1 for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            ischemic_profile = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))
        ischemic_profile.SetName("IschemicProfile")
        ray_projected.GetPointData().AddArray(ischemic_profile)

        territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
        territory = territory_profile[territory_profile > 0]
        if territory.size > 0:
            territory_tag = np.bincount(territory).argmax()
            territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
        territory_profile.SetName("TerritoryProfile")
        ray_projected.GetPointData().AddArray(territory_profile)

        all_ray.AddInputData(BoldLine(ray_projected))
    all_ray.Update()
    Upper_Rays.AddInputData(all_ray.GetOutput())
    #line_path = f"{OutputFolder}/ray_projected_upper{k}.vtp"
    #WriteVTPFile(line_path, all_ray.GetOutput())
Upper_Rays.Update()
#line_path = f"{OutputFolder}/ray_upper.vtu"
#WriteVTUFile(line_path, Upper_Rays.GetOutput())

MBF_Upper_Ischemic_Profile = ProbeMyocardium(MBF_Upper_Surface, Upper_Rays.GetOutput()) 

#WriteVTPFile(cylinder_lower, MBF_Lower_Ischemic_Profile)
#WriteVTPFile(cylinder_upper, MBF_Upper_Ischemic_Profile)


Myocardium_Surface = vtk.vtkAppendPolyData()
Myocardium_Surface.AddInputData(MBF_Upper_Ischemic_Profile)
Myocardium_Surface.AddInputData(MBF_Lower_Ischemic_Profile)
Myocardium_Surface.Update()
surface = f"{OutputFolder}/MBF_surface.vtp"
WriteVTPFile(surface, Myocardium_Surface.GetOutput())

In [ ]:
def ExtractSurface(volume):
	#Get the outer surface of the volume
	surface=vtk.vtkDataSetSurfaceFilter()
	surface.SetInputData(volume)
	surface.Update()
	return surface.GetOutput()

surface = f"{OutputFolder}/MBF_surface.vtp"
WriteVTPFile(surface, ExtractSurface(MBF_Lower))

In [ ]:
def Cylinder(MyocardiumCenterLine):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(MyocardiumCenterLine)
    tube_filter.SetRadius(35)  # Adjust this value to change thickness
    tube_filter.SetNumberOfSides(50)  # Higher = smoother tube
    tube_filter.CappingOff()  # Close tube ends
    tube_filter.Update()

    return tube_filter.GetOutput()

cylinder = f"{OutputFolder}/Cylinder.vtp"
WriteVTPFile(cylinder, Cylinder(CL_str_lower))

In [ ]:
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk

mbf_lower = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/RayBasedIschemicProfile_SplitMyocardium/MBF_Lower.vtu"
mbf_upper = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/RayBasedIschemicProfile_SplitMyocardium/MBF_Upper.vtu"

MBF_Lower = ReadVTU(mbf_lower)
MBF_Upper = ReadVTU(mbf_upper)

def ExtractSurface(volume):
	#Get the outer surface of the volume
	surface=vtk.vtkDataSetSurfaceFilter()
	surface.SetInputData(volume)
	surface.Update()
	return surface.GetOutput()

MBF_Lower_Surface = ExtractSurface(MBF_Lower)
MBF_Upper_Surface = ExtractSurface(MBF_Upper)


def PCA_Analysis(Volume):
    points = Volume.GetPoints()
    point_coords = np.array([points.GetPoint(i) for i in range(points.GetNumberOfPoints())])

    mean_center = np.mean(point_coords, axis=0)
    cov_matrix = np.cov(point_coords.T)
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

    sorted_indices = np.argsort(-eigenvalues)
    long_axis = eigenvectors[:, sorted_indices[0]]
    short_axis1 = eigenvectors[:, sorted_indices[1]]
    short_axis2 = eigenvectors[:, sorted_indices[2]]

    return mean_center, long_axis, short_axis1, short_axis2

center_lower, _, _, short_axis_lower = PCA_Analysis(MBF_Lower)
center_upper, _, _, short_axis_upper = PCA_Analysis(MBF_Upper)

length = 35

point1 = np.array([center_lower[0] + short_axis_lower[0]*length, center_lower[1] + short_axis_lower[1]*length, center_lower[2] + short_axis_lower[2]*length])
point2 = np.array([center_lower[0] - short_axis_lower[0]*length, center_lower[1] - short_axis_lower[1]*length, center_lower[2] - short_axis_lower[2]*length])

res = 50
CL_str_lower = Line(point1, point2, res)
OutputFolder = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/RayBasedIschemicProfile_SplitMyocardium"

point1 = np.array([center_upper[0] + short_axis_upper[0]*length, center_upper[1] + short_axis_upper[1]*length, center_upper[2] + short_axis_upper[2]*length])
point2 = np.array([center_upper[0] - short_axis_upper[0]*length, center_upper[1] - short_axis_upper[1]*length, center_upper[2] - short_axis_upper[2]*length])

CL_str_upper = Line(point1, point2, res)

WriteVTPFile(f"{OutputFolder}/CL_PCA_Lower.vtp", BoldLine(CL_str_lower))
WriteVTPFile(f"{OutputFolder}/CL_PCA_Upper.vtp", BoldLine(CL_str_upper))

points_lower = CL_str_lower.GetPoints()
center_points_lower = np.array([points_lower.GetPoint(i) for i in range(points_lower.GetNumberOfPoints())])

points_upper = CL_str_upper.GetPoints()
center_points_upper = np.array([points_upper.GetPoint(i) for i in range(points_upper.GetNumberOfPoints())])




def Cylinder(MyocardiumCenterLine):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(MyocardiumCenterLine)
    tube_filter.SetRadius(35)  # Adjust this value to change thickness
    tube_filter.SetNumberOfSides(3600)  # Higher = smoother tube
    tube_filter.CappingOff()  # Close tube ends
    tube_filter.Update()
    return tube_filter.GetOutput()

#cylinder_lower = f"{OutputFolder}/CylinderLower.vtp"
Cylinder_Lower = Cylinder(CL_str_lower)
Cylinder_Upper = Cylinder(CL_str_upper)


Cylinder_Lower_vtu = vtk.vtkAppendFilter()
Cylinder_Lower_vtu.AddInputData(Cylinder_Lower)
Cylinder_Lower_vtu.Update()

Cylinder_Upper_vtu = vtk.vtkAppendFilter()
Cylinder_Upper_vtu.AddInputData(Cylinder_Upper)
Cylinder_Upper_vtu.Update()



ray_lower_ = np.array([-short_axis_lower[2], 0, short_axis_lower[0]])
n_ray = 1800
res = 50
angles = np.linspace(0, 360, n_ray, endpoint= False)
Lower_Rays = vtk.vtkAppendPolyData()
for k in range(len(center_points_lower)):
    origin = center_points_lower[k]
    all_ray = vtk.vtkAppendPolyData()
    for i in range(n_ray):
        ray_new = rotate_vector(ray_lower_, short_axis_lower, angles[i])
        point2 = np.array([origin[0]+ 100*ray_new[0], origin[1] + 100*ray_new[1], origin[2] + 100*ray_new[2]])
        ray = Line(origin, point2, res)
        ray_projected = ProbeMyocardium(ray, MBF_Lower)
        ischemic_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("IschemicTerritory"))
        is_ischemic = np.where(ischemic_profile > 0.5)[0]
        if is_ischemic.size > 0:
            ischemic_profile = numpy_to_vtk(np.array([1 for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            ischemic_profile = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))
        ischemic_profile.SetName("IschemicProfile")
        ray_projected.GetPointData().AddArray(ischemic_profile)

        territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
        territory = territory_profile[territory_profile > 0]
        if territory.size > 0:
            territory_tag = np.bincount(territory).argmax()
            territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
        territory_profile.SetName("TerritoryProfile")
        ray_projected.GetPointData().AddArray(territory_profile)
        
        all_ray.AddInputData(ray_projected)
    all_ray.Update()
    Lower_Rays.AddInputData(all_ray.GetOutput())
Lower_Rays.Update()
#line_path = f"{OutputFolder}/ray_lower.vtp"
#WriteVTPFile(line_path, Lower_Rays.GetOutput())
MBF_Lower_Ischemic_Profile = ProbeMyocardium(Cylinder_Lower_vtu.GetOutput(), Lower_Rays.GetOutput()) 

cylinder_lower = f"{OutputFolder}/Cylinder_Lower.vtu"
WriteVTUFile(cylinder_lower, MBF_Lower_Ischemic_Profile)



In [ ]:
ray_upper_ = np.array([-short_axis_upper[2], 0, short_axis_upper[0]])
angles = np.linspace(0, 360, n_ray, endpoint= False)
Upper_Rays = vtk.vtkAppendPolyData()
for k in range(len(center_points_upper)):
    origin = center_points_upper[k]
    all_ray = vtk.vtkAppendPolyData()
    for i in range(n_ray):
        ray_new = rotate_vector(ray_upper_, short_axis_upper, angles[i])
        point2 = np.array([origin[0]+ 100*ray_new[0], origin[1] + 100*ray_new[1], origin[2] + 100*ray_new[2]])
        ray = Line(origin, point2, res)
        ray_projected = ProbeMyocardium(ray, MBF_Upper)
        ischemic_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("IschemicTerritory"))
        is_ischemic = np.where(ischemic_profile > 0.5)[0]
        if is_ischemic.size > 0:
            ischemic_profile = numpy_to_vtk(np.array([1 for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            ischemic_profile = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))
        ischemic_profile.SetName("IschemicProfile")
        ray_projected.GetPointData().AddArray(ischemic_profile)

        territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
        territory = territory_profile[territory_profile > 0]
        if territory.size > 0:
            territory_tag = np.bincount(territory).argmax()
            territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
        territory_profile.SetName("TerritoryProfile")
        ray_projected.GetPointData().AddArray(territory_profile)

        all_ray.AddInputData(BoldLine(ray_projected))
    all_ray.Update()
    Upper_Rays.AddInputData(all_ray.GetOutput())
    #line_path = f"{OutputFolder}/ray_projected_upper{k}.vtp"
    #WriteVTPFile(line_path, all_ray.GetOutput())
Upper_Rays.Update()
#line_path = f"{OutputFolder}/ray_upper.vtu"
#WriteVTUFile(line_path, Upper_Rays.GetOutput())

MBF_Upper_Ischemic_Profile = ProbeMyocardium(Cylinder_Upper_vtu.GetOutput(), Upper_Rays.GetOutput()) 

cylinder_upper = f"{OutputFolder}/Cylinder_Upper.vtu"
#WriteVTPFile(cylinder_lower, MBF_Lower_Ischemic_Profile)
WriteVTUFile(cylinder_upper, MBF_Upper_Ischemic_Profile)


Myocardium_Surface = vtk.vtkAppendFilter()
Myocardium_Surface.AddInputData(MBF_Upper_Ischemic_Profile)
Myocardium_Surface.AddInputData(MBF_Lower_Ischemic_Profile)
Myocardium_Surface.Update()
surface = f"{OutputFolder}/MBF_surface.vtu"
WriteVTUFile(surface, Myocardium_Surface.GetOutput())

In [ ]:
is_Myo_arr = np.zeros(points_lower.GetNumberOfPoints())
for i in range(points_lower.GetNumberOfPoints()):
    Myo_slice = CutPlane(MBF_Lower, center_points_lower[i], short_axis_lower)
    ImageScalars = vtk_to_numpy(Myo_slice.GetPointData().GetArray("ImageScalars"))
    
    if ImageScalars.size > 0:
        is_Myo_arr[i] = 1
    
is_Myo_arr = numpy_to_vtk(is_Myo_arr)
is_Myo_arr.SetName("MyocardiumProfile")
CL_str_lower.GetPointData().AddArray(is_Myo_arr)




is_Myo_arr = np.zeros(points_lower.GetNumberOfPoints())
for i in range(points_upper.GetNumberOfPoints()):
    Myo_slice = CutPlane(MBF_Upper, center_points_upper[i], short_axis_upper)
    ImageScalars = vtk_to_numpy(Myo_slice.GetPointData().GetArray("ImageScalars"))
    
    if ImageScalars.size > 0:
        is_Myo_arr[i] = 1
    
is_Myo_arr = numpy_to_vtk(is_Myo_arr)
is_Myo_arr.SetName("MyocardiumProfile")
CL_str_upper.GetPointData().AddArray(is_Myo_arr)

WriteVTPFile(f"{OutputFolder}/CL_PCA_Lower.vtp", BoldLine(CL_str_lower))
WriteVTPFile(f"{OutputFolder}/CL_PCA_Upper.vtp", BoldLine(CL_str_upper))


In [ ]:
import vtk
import glob
import argparse
import numpy as np
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
from utilities import ReadVTUFile, ReadVTPFile, WriteVTPFile, GetCentroid, ThresholdByUpper, ThresholdInBetween, WriteVTUFile, LargestConnectedRegion, PrintProgress


def Line(point1, point2, res):
    line = vtk.vtkLineSource()
    line.SetPoint1(point1)
    line.SetPoint2(point2)
    line.SetResolution(res)
    line.Update()
        
    return line.GetOutput()

InputFolder = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/RayBasedIschemicProfile_2"
InputMBFBase = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/MERGE/Myocardium.vtu"
InputMBFApex = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/MERGE/Apex.vtu"
InputIschemic = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/MERGE/MBFIschemicRegion_cutoff90_Dg.vtu"
NSection = 80
NRays = 1000
NRaySphere = 10000
TerritoryTag = ['Diag1_0', 'Diag1_2', 'Diag1_3', 'Diag1_4']

slice_apex = f"{InputFolder}/Slice_Apex.vtp"
slice_base = f"{InputFolder}/Slice_Base.vtp"

SliceApex = ReadVTPFile(slice_apex)
SliceBase = ReadVTPFile(slice_base)

base_centeroid = GetCentroid(SliceBase)
apex_centeroid = GetCentroid(SliceApex)

AnnotationPoints = [base_centeroid, apex_centeroid]

centerline_axis = np.array([AnnotationPoints[1][0] - AnnotationPoints[0][0], 
                                AnnotationPoints[1][1] - AnnotationPoints[0][1], 
                                AnnotationPoints[1][2] - AnnotationPoints[0][2]])

CL_axis = centerline_axis/np.linalg.norm(centerline_axis)

MBF = ReadVTUFile(InputMBFBase)
Apex = ReadVTUFile(InputMBFApex)

"""
Myocardium_centeroid = GetCentroid(self.MBF)
point1 = np.array([Myocardium_centeroid[0] + self.CL_axis[0]*60, Myocardium_centeroid[1] + self.CL_axis[1]*60, Myocardium_centeroid[2] + self.CL_axis[2]*60])
point2 = np.array([Myocardium_centeroid[0] - self.CL_axis[0]*50, Myocardium_centeroid[1] - self.CL_axis[1]*50, Myocardium_centeroid[2] - self.CL_axis[2]*50])
"""

Ischemic = LargestConnectedRegion(ReadVTUFile(InputIschemic))

CenterLine = Line(AnnotationPoints[0], AnnotationPoints[1], NSection)


def BoldLine(centerline):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(centerline)
    tube_filter.SetRadius(0.2)  # Adjust this value to change thickness
    tube_filter.SetNumberOfSides(50)  # Higher = smoother tube
    tube_filter.CappingOn()  # Close tube ends
    tube_filter.Update()
    
    return tube_filter.GetOutput()

def SliceWPlane(Volume,Origin,Norm):
    plane=vtk.vtkPlane()
    plane.SetOrigin(Origin)
    plane.SetNormal(Norm)
    Slice=vtk.vtkCutter()
    Slice.GenerateTrianglesOff()
    Slice.SetCutFunction(plane)
    Slice.SetInputData(Volume)
    Slice.Update()
    
    return Slice.GetOutput()

def ClipWPlane(surface,center, axis):
    plane = vtk.vtkPlane()
    plane.SetOrigin(center)
    plane.SetNormal(axis)

    clipper = vtk.vtkClipPolyData()
    clipper.SetInputData(surface)
    clipper.SetClipFunction(plane)
    clipper.InsideOutOff()
    #clipper.GetOutputInformation(1)
    clipper.Update()

    return clipper.GetOutput()


def RotateVector(ray, normal, angle):
    angle = np.radians(angle)
    normal /= np.linalg.norm(normal)
    ray_rot = (ray*np.cos(angle) + np.cross(normal, ray) * np.sin(angle) + normal * np.dot(normal, ray) * (1 - np.cos(angle)))
    
    return ray_rot

def ProbeFilter(InputData, SourceData):
    probe = vtk.vtkProbeFilter()
    probe.AddInputData(InputData)
    probe.SetSourceData(SourceData)
    probe.Update()
    
    return probe.GetOutput()

Radius = 35
def Cylinder():
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(CenterLine)
    tube_filter.SetRadius(35)  # Adjust this value to change thickness
    tube_filter.SetNumberOfSides(2*NRay)  # Higher = smoother tube
    tube_filter.CappingOff()  # Close tube ends
    tube_filter.Update()
    
    return tube_filter.GetOutput()

def Hemisphere():
    sphere = vtk.vtkSphereSource()
    sphere.SetCenter(apex_centeroid)
    sphere.SetRadius(Radius)
    sphere.SetPhiResolution(4*NRaySphere)
    sphere.SetThetaResolution(4*NRaySphere)
    sphere.Update()

    Hemisphere = ClipWPlane(sphere.GetOutput(), apex_centeroid, CL_axis)

    return Hemisphere

def fibonacci_sphere(samples=100):
    """Generates evenly distributed unit vectors over a sphere"""
    points = []
    phi = np.pi * (3. - np.sqrt(5.))  # Golden angle in radians

    for i in range(samples):
        y = 1 - (i / float(samples - 1)) * 2  # y goes from 1 to -1
        radius = np.sqrt(1 - y * y)  # Radius at given y
        theta = phi * i  # Golden angle increment

        x = np.cos(theta) * radius
        z = np.sin(theta) * radius
        points.append([x, y, z])

    return np.array(points)

def ThresholdPointsByUpper(PolyData,arrayname,value):
	Threshold=vtk.vtkThresholdPoints()
	Threshold.SetInputData(PolyData)
	Threshold.ThresholdByUpper(value)
	Threshold.SetInputArrayToProcess(0,0,0,"vtkDataObject::FIELD_ASSOCIATION_POINTS",arrayname)
	Threshold.Update()
	return Threshold.GetOutput()


def CastRaysVisualizations(CLPoints):
    #>>> Ray Casting across the Myocardium Cylinder 
        slices = vtk.vtkAppendPolyData()

        NRays_sample = 20
        ray_ = np.array([-CL_axis[2], 0, CL_axis[0]])
        res = 50
        angles = np.linspace(0, 360, NRays_sample, endpoint= False)
        Rays = vtk.vtkAppendPolyData()
        
        for k in range(0,len(CLPoints), int(len(CLPoints)/(NSection/10))):
            Myo_slice = SliceWPlane(MBF, CLPoints[k], CL_axis)
            slices.AddInputData(Myo_slice)

            origin = CLPoints[k]
            slice_rays = vtk.vtkAppendPolyData()
            for i in range(NRays_sample):
                ray_new = RotateVector(ray_, CL_axis, angles[i])
                point2 = np.array([origin[0]+ 100*ray_new[0], origin[1] + 100*ray_new[1], origin[2] + 100*ray_new[2]])
                ray = Line(origin, point2, res)
                ray_projected = ProbeFilter(ray, Ischemic)
                
                ischemic_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("ImageScalars"))
                is_ischemic = ischemic_profile[ischemic_profile > 0]
                if is_ischemic.size > 0:
                    ischemic_profile = numpy_to_vtk(np.array([1 for _ in range(ray_projected.GetNumberOfPoints())]))
                else:
                    ischemic_profile = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))

                ray_projected = ProbeFilter(ray_projected, MBF)
                
                ischemic_profile.SetName("IschemicProfile")
                ray_projected.GetPointData().AddArray(ischemic_profile)

                territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
                territory = territory_profile[territory_profile > 0]
                
                if territory.size > 0:
                    territory_tag = np.bincount(territory).argmax()
                    territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
                else:
                    territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
                
                territory_profile.SetName("TerritoryProfile")
                ray_projected.GetPointData().AddArray(territory_profile)
                slice_rays.AddInputData(BoldLine(ray_projected))
        
            slice_rays.Update()
        
            Rays.AddInputData(slice_rays.GetOutput())


        #>>> Ray Casting across the Apex Hemisphere
        NRays_sample = 100
        directions = fibonacci_sphere(NRays_sample)
        origin = apex_centeroid
        Rays_ = vtk.vtkAppendPolyData()

        for i in range(NRays_sample):
            ray_new = directions[i]
            point2 = np.array([origin[0]+ 50*ray_new[0], origin[1] + 50*ray_new[1], origin[2] + 50*ray_new[2]])
            ray = Line(origin, point2, res)
            ray_projected = ProbeFilter(ray, Ischemic)
            
            ischemic_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("ImageScalars"))
            is_ischemic = ischemic_profile[ischemic_profile > 0]
            if is_ischemic.size > 0:
                ischemic_profile = numpy_to_vtk(np.array([1 for _ in range(ray_projected.GetNumberOfPoints())]))
            else:
                ischemic_profile = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))

            ray_projected = ProbeFilter(ray_projected, Apex)
            
            ischemic_profile.SetName("IschemicProfile")
            ray_projected.GetPointData().AddArray(ischemic_profile)

            territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
            territory = territory_profile[territory_profile > 0]
            
            if territory.size > 0:
                territory_tag = np.bincount(territory).argmax()
                territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
            else:
                territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
            
            territory_profile.SetName("TerritoryProfile")
            ray_projected.GetPointData().AddArray(territory_profile)
            Rays_.AddInputData(BoldLine(ray_projected))


        Rays_.Update()
        Rays_ = ClipWPlane(Rays_.GetOutput(), apex_centeroid, CL_axis)
        Rays.AddInputData(Rays_)
        Rays.Update()
        ray_path = f"{InputFolder}/Rays.vtp"
        WriteVTPFile(ray_path, Rays.GetOutput())
    

        slices.Update()
        slice_path = f"{InputFolder}/MyoSlices.vtp"
        WriteVTPFile(slice_path, slices.GetOutput())

def RayCasting(CLPoints):

    
    ray_ = np.array([-CL_axis[2], 0, CL_axis[0]])
    res = 50
    angles = np.linspace(0, 360, NRays, endpoint= False)
    Rays = vtk.vtkAppendPolyData()
    
    print("------ Ray Casting across Sections along Myocardium:")
    progress_old = -1
    for k in range(0,len(CLPoints)):
        progress = PrintProgress(k,len(CLPoints),progress_old)
        progress_old = progress

        origin = CLPoints[k]
        #slice_rays = vtk.vtkAppendPolyData()
        
        for i in range(NRays):
            ray_new = RotateVector(ray_, CL_axis, angles[i])
            point2 = np.array([origin[0]+ 100*ray_new[0], origin[1] + 100*ray_new[1], origin[2] + 100*ray_new[2]])
            ray = Line(origin, point2, res)
            ray_projected = ProbeFilter(ray, Ischemic)
            
            ischemic_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("ImageScalars"))
            is_ischemic = ischemic_profile[ischemic_profile > 0]
            if is_ischemic.size > 0:
                ischemic_profile = numpy_to_vtk(np.array([1 for _ in range(ray_projected.GetNumberOfPoints())]))
            else:
                ischemic_profile = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))

            ray_projected = ProbeFilter(ray_projected, MBF)
            
            ischemic_profile.SetName("IschemicProfile")
            ray_projected.GetPointData().AddArray(ischemic_profile)

            territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
            territory = territory_profile[territory_profile > 0]
            
            if territory.size > 0:
                territory_tag = np.bincount(territory).argmax()
                territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
            else:
                territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
            
            territory_profile.SetName("TerritoryProfile")
            ray_projected.GetPointData().AddArray(territory_profile)
            Rays.AddInputData(BoldLine(ray_projected))
    
        #slice_rays.Update()
    
        #Rays.AddInputData(slice_rays.GetOutput())
        Rays.Update()

    directions = fibonacci_sphere(NRaySphere)
    origin = apex_centeroid
    Rays_Hemisphere = vtk.vtkAppendPolyData()

    print("------ Ray Casting across the Apex Hemisphere:")
    progress_old = -1
    for i in range(NRaySphere):
        progress = PrintProgress(i, NRaySphere, progress_old)
        progress_old = progress

        ray_new = directions[i]
        point2 = np.array([origin[0]+ 50*ray_new[0], origin[1] + 50*ray_new[1], origin[2] + 50*ray_new[2]])
        ray = Line(origin, point2, res)
        ray_projected = ProbeFilter(ray, Ischemic)
        
        ischemic_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("ImageScalars"))
        is_ischemic = ischemic_profile[ischemic_profile > 0]
        if is_ischemic.size > 0:
            ischemic_profile = numpy_to_vtk(np.array([1 for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            ischemic_profile = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))

        ray_projected = ProbeFilter(ray_projected, Apex)
        
        ischemic_profile.SetName("IschemicProfile")
        ray_projected.GetPointData().AddArray(ischemic_profile)

        territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
        territory = territory_profile[territory_profile > 0]
        
        if territory.size > 0:
            territory_tag = np.bincount(territory).argmax()
            territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
        else:
            territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
        
        territory_profile.SetName("TerritoryProfile")
        ray_projected.GetPointData().AddArray(territory_profile)
        Rays_Hemisphere.AddInputData(BoldLine(ray_projected))


    Rays_Hemisphere.Update()
    #print("------ Clipping Sphere")
    #Rays_ = self.ClipWPlane(Rays_.GetOutput(), self.apex_centeroid, self.CL_axis)
    

    return Rays.GetOutput(), Rays_Hemisphere.GetOutput()

import os 

def SpatialCorrespondance(Cylinder_Projected):
    with open(f"{InputFolder}/MBF_Territories_Labels.dat",'r') as infile:
        infile.readline()
        TerritoryLabels=[]
        TerritoryNames = ""
        for LINE in infile:
            line=LINE.split()
            for tag in TerritoryTag:
                if line[1].find(tag)>=0: 
                    TerritoryLabels.append(int(line[0]))
                    TerritoryNames += tag + "_"
    print(TerritoryNames)

    ThresholdArray = np.zeros(Cylinder_Projected.GetNumberOfPoints())
    for i in range(Cylinder_Projected.GetNumberOfPoints()):
        if int(Cylinder_Projected.GetPointData().GetArray("TerritoryProfile").GetValue(i)) in TerritoryLabels:
            ThresholdArray[i] = 1
    
    ThresholdArrayVTK = numpy_to_vtk(ThresholdArray, deep=True)
    ThresholdArrayVTK.SetName(f"TerritoryLabels_{TerritoryNames}")
    Cylinder_Projected.GetPointData().AddArray(ThresholdArrayVTK)
    Cylinder_Projected.Modified()

    TerritoryRegions = ThresholdPointsByUpper(Cylinder_Projected, f"TerritoryLabels_{TerritoryNames}", 1)
    WriteVTPFile(InputFolder + f"/Projected_Territories_{TerritoryNames}.vtp", TerritoryRegions)

    IschemicRegions = ThresholdPointsByUpper(Cylinder_Projected, "IschemicProfile", 1)
    ischemic_surface_path = os.path.splitext(os.path.basename(InputIschemic))[0]
    WriteVTPFile(InputFolder + f"/{ischemic_surface_path}.vtp", IschemicRegions)

    TerritoryRegionsPoints=vtk_to_numpy(TerritoryRegions.GetPoints().GetData())
    IschemicRegionsPoints=vtk_to_numpy(IschemicRegions.GetPoints().GetData())


    TerritoryTesselationString=[]
    TerritoryIschemicString=[]
    for i in range(TerritoryRegions.GetNumberOfPoints()): TerritoryTesselationString.append(str(TerritoryRegionsPoints[i]))
    for i in range(IschemicRegions.GetNumberOfPoints()): TerritoryIschemicString.append(str(IschemicRegionsPoints[i]))
    TotalPointsString=TerritoryTesselationString+TerritoryIschemicString
    UniquePointsString=set(TotalPointsString)
    OverlapCounter=len(TotalPointsString)-len(UniquePointsString)

    DiceScore = (2*OverlapCounter)/(TerritoryRegions.GetNumberOfPoints() + IschemicRegions.GetNumberOfPoints())
    print(DiceScore)
    return Cylinder_Projected


In [ ]:

print("--- Extracting the Centerline of the Myocardium")
cl_file = f"{InputFolder}/CenterLine.vtp"
WriteVTPFile(cl_file, BoldLine(CenterLine))

print("--- Visualizing Ray Casting")
CLPoints = CenterLine.GetPoints()
CLPointsArray = np.array([CLPoints.GetPoint(i) for i in range(CLPoints.GetNumberOfPoints())])
CastRaysVisualizations(CLPointsArray)

print("--- Ray Casting Across the Myocardium")
Rays_Myocardium, Rays_Hemisphere = RayCasting(CLPointsArray)

print("--- Creating the Output Surface")
Cylinder = Cylinder()
Hemisphere = Hemisphere()

print("--- Projecting the Rays onto the Output Surface")
#OSurface_Projected = self.ProbeFilter(OutputSurface.GetOutput(), Rays)
Cylinder_Projected = ProbeFilter(Cylinder, Rays_Myocardium)
Hemisphere_Projected = ProbeFilter(Hemisphere, Rays_Hemisphere)

print("--- Writing the Output Surface")
OutputSurface = vtk.vtkAppendPolyData()
OutputSurface.AddInputData(Cylinder_Projected)
OutputSurface.AddInputData(Hemisphere_Projected)
OutputSurface.Update()


print("--- Calculating the Spatial Correspondeance")
OSurface_Projected = SpatialCorrespondance(OutputSurface.GetOutput())

output_path = f"{InputFolder}/Output_Projected.vtp"
WriteVTPFile(output_path, OSurface_Projected)


--- Extracting the Centerline of the Myocardium
--- Visualizing Ray Casting
--- Ray Casting Across the Myocardium
--- Calculating the Spatial Correspondeance


In [3]:
cylinder_path = f"{InputFolder}/Cylinder.vtp"
WriteVTPFile(cylinder_path, Cylinder_Projected)

In [4]:
with open(f"{InputFolder}/MBF_Territories_Labels.dat",'r') as infile:
    infile.readline()
    TerritoryLabels=[]
    TerritoryNames = ""
    for LINE in infile:
        line=LINE.split()
        for tag in TerritoryTag:
            if line[1].find(tag)>=0: 
                TerritoryLabels.append(int(line[0]))
                TerritoryNames += tag + "_"
print(TerritoryNames)
print(TerritoryLabels)


Diag1_0_Diag1_2_Diag1_3_Diag1_4_
[12, 14, 15, 16]


In [5]:
def ThresholdPointsByUpper(PolyData,arrayname,value):
	Threshold=vtk.vtkThresholdPoints()
	Threshold.SetInputData(PolyData)
	Threshold.ThresholdByUpper(value)
	Threshold.SetInputArrayToProcess(0,0,0,"vtkDataObject::FIELD_ASSOCIATION_POINTS",arrayname)
	Threshold.Update()
	return Threshold.GetOutput()




ThresholdArray = np.zeros(Cylinder_Projected.GetNumberOfPoints())
for i in range(Cylinder_Projected.GetNumberOfPoints()):
    if int(Cylinder_Projected.GetPointData().GetArray("TerritoryProfile").GetValue(i)) in TerritoryLabels:
        ThresholdArray[i] = 1

ThresholdArrayVTK = numpy_to_vtk(ThresholdArray, deep=True)
ThresholdArrayVTK.SetName(f"TerritoryLabels_{TerritoryNames}")
Cylinder_Projected.GetPointData().AddArray(ThresholdArrayVTK)
Cylinder_Projected.Modified()

'''
ThresholdPoints = vtk.vtkPoints()
for i in range(ThresholdArrayVTK.GetNumberOfTuples()):
    x, y, z = ThresholdArrayVTK.GetTuple3(i)
    ThresholdPoints.InsertNextPoint(x, y, z)

TerritoryRegions = vtk.vtkPolyData()
TerritoryRegions.SetPoints(ThresholdPoints)
'''


TerritoryRegions = ThresholdPointsByUpper(Cylinder_Projected, f"TerritoryLabels_{TerritoryNames}", 0.1)

#print(ThresholdArrayVTK)
#print(TerritoryRegions)
WriteVTPFile(InputFolder + f"/Projected_Territories_{TerritoryNames}.vtp", TerritoryRegions)

cylinder_path = f"{InputFolder}/Cylinder_Territory.vtp"
WriteVTPFile(cylinder_path, Cylinder_Projected)

In [6]:
import os

IschemicRegions = ThresholdPointsByUpper(Cylinder_Projected, "IschemicProfile", 1)
ischemic_surface_path = os.path.splitext(os.path.basename(InputIschemic))[0]
WriteVTPFile(InputFolder + f"/{ischemic_surface_path}.vtp", IschemicRegions)

TerritoryRegionsPoints=vtk_to_numpy(TerritoryRegions.GetPoints().GetData())
IschemicRegionsPoints=vtk_to_numpy(IschemicRegions.GetPoints().GetData())


TerritoryTesselationString=[]
TerritoryIschemicString=[]
for i in range(TerritoryRegions.GetNumberOfPoints()): TerritoryTesselationString.append(str(TerritoryRegionsPoints[i]))
for i in range(IschemicRegions.GetNumberOfPoints()): TerritoryIschemicString.append(str(IschemicRegionsPoints[i]))
TotalPointsString=TerritoryTesselationString+TerritoryIschemicString
UniquePointsString=set(TotalPointsString)
OverlapCounter=len(TotalPointsString)-len(UniquePointsString)

DiceScore = (2*OverlapCounter)/(TerritoryRegions.GetNumberOfPoints() + IschemicRegions.GetNumberOfPoints())
print(DiceScore)



0.7599376407413823


In [7]:
apex = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/RayBasedIschemicProfile_2/Apex.vtu"
Apex = ReadVTUFile(apex)

apex_slice = "/Users/ana/Documents/AnahitaSeresti/Tesselation/MBFAnalysis/SU03A/RayBasedIschemicProfile_2/ApexSlice.vtp"
ApexSlice = ReadVTPFile(apex_slice)

In [ ]:
apex_centeroid = GetCentroid(ApexSlice)


sphere = vtk.vtkSphereSource()
sphere.SetCenter(apex_centeroid)
sphere.SetRadius(20)
sphere.SetPhiResolution(50)
sphere.SetThetaResolution(50)
sphere.Update()



Sphere_Apex = sphere.GetOutput()

sphere_path = InputFolder + "/ApexSphere.vtp"
WriteVTPFile(sphere_path, Sphere_Apex)



directions = fibonacci_sphere(10000)

NRays_sample = 10000
#ray_ = np.array([-CL_axis[2], 0, CL_axis[0]])
res = 50
origin = apex_centeroid
Rays = vtk.vtkAppendPolyData()

for i in range(NRays_sample):
    ray_new = directions[i]
    point2 = np.array([origin[0]+ 50*ray_new[0], origin[1] + 50*ray_new[1], origin[2] + 50*ray_new[2]])
    ray = Line(origin, point2, res)
    ray_projected = ProbeFilter(ray, Ischemic)
    
    ischemic_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("ImageScalars"))
    is_ischemic = ischemic_profile[ischemic_profile > 0]
    if is_ischemic.size > 0:
        ischemic_profile = numpy_to_vtk(np.array([1 for _ in range(ray_projected.GetNumberOfPoints())]))
    else:
        ischemic_profile = numpy_to_vtk(np.array([0 for _ in range(ray_projected.GetNumberOfPoints())]))

    ray_projected = ProbeFilter(ray_projected, MBF)
    
    ischemic_profile.SetName("IschemicProfile")
    ray_projected.GetPointData().AddArray(ischemic_profile)

    territory_profile = vtk_to_numpy(ray_projected.GetPointData().GetArray("TerritoryMaps"))
    territory = territory_profile[territory_profile > 0]
    
    if territory.size > 0:
        territory_tag = np.bincount(territory).argmax()
        territory_profile = numpy_to_vtk(np.array([territory_tag for _ in range(ray_projected.GetNumberOfPoints())]))
    else:
        territory_profile = numpy_to_vtk(np.array([-1 for _ in range(ray_projected.GetNumberOfPoints())]))
    
    territory_profile.SetName("TerritoryProfile")
    ray_projected.GetPointData().AddArray(territory_profile)
    Rays.AddInputData(BoldLine(ray_projected))

Rays.Update()


Rays_hemisphere = PlaneClip(Rays.GetOutput(), apex_centeroid, CL_axis)

ray_path = f"{InputFolder}/Rays_spherical.vtp"
WriteVTPFile(ray_path, Rays_hemisphere)

In [ ]:
def PlaneClip(surface,center, axis):
    plane = vtk.vtkPlane()
    plane.SetOrigin(center)
    plane.SetNormal(axis)

    clipper = vtk.vtkClipPolyData()
    clipper.SetInputData(surface)
    clipper.SetClipFunction(plane)
    clipper.InsideOutOff()
    #clipper.GetOutputInformation(1)
    clipper.Update()

    return clipper.GetOutput()




In [ ]:

Rays_hemisphere = PlaneClip(Rays.GetOutput(), apex_centeroid, CL_axis)

ray_path = f"{InputFolder}/Rays_spherical.vtp"
WriteVTPFile(ray_path, Rays_hemisphere)

